In [67]:
import math
import numpy as np

In [68]:
def number_of_stubs (nodes_per_prob):
    stubs = 0
    for i in range(0, len(nodes_per_prob)):
        stubs += (i + 1) * nodes_per_prob[i]
    return stubs

def poisson (z: int, k: int) -> float:
    return ((math.pow(z, k)) * (math.exp(-z))) / math.factorial(k)

def generate_degree_sequence (z: int, population: int, max_degree: int):
    probabilities = [poisson(z, k) for k in range(1, max_degree)]
    nodes_per_prob = [math.ceil(p * 1000) for p in probabilities]
    print(f'Stubs Before = {number_of_stubs(nodes_per_prob)}')
    print(f'Nodes left to populate = {population - np.sum(nodes_per_prob)}')
    nodes_left = population - np.sum(nodes_per_prob)
    average_degree = round(np.sum([k * ((math.pow(z, k)) * (math.exp(-z))) / math.factorial(k) for k in range(1, max_degree)]))
    while nodes_left > 0:
        if (nodes_left == 1):
            if (number_of_stubs(nodes_per_prob) + average_degree) % 2 == 0:
                # No stubs left over
                nodes_per_prob[average_degree - 1] += 1
            else:
                # 1 Stub left over, add 1 to average degree
                nodes_per_prob[average_degree] += 1
        else:
            nodes_per_prob[average_degree - 1] += 1 # Add the node into the distribution
        nodes_left -= 1
        
    return nodes_per_prob

In [69]:
z = 4
population = 1500
max_degree = 20

degree_sequence = generate_degree_sequence(z, population, max_degree)

for k in range(1, max_degree):
    print(f'{k}: {degree_sequence[k - 1]}')

Stubs Before = 4147
Nodes left to populate = 505
1: 74
2: 147
3: 196
4: 700
5: 158
6: 105
7: 60
8: 30
9: 14
10: 6
11: 2
12: 1
13: 1
14: 1
15: 1
16: 1
17: 1
18: 1
19: 1


In [70]:
np.sum(degree_sequence)

1500

In [71]:
number_of_stubs(degree_sequence) % 2 == 0

True